In [2]:
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, RepeatedKFold

In [4]:
# train_df = pd.read_csv('../feature_data/train/featureInput_noChampionRole_ZSCORED.csv')
train_df = pd.read_csv('trial_train_data2.csv')
# test_df = pd.read_csv('../feature_data/test/featureInput_noChampionRole_ZSCORED.csv')
test_df = pd.read_csv('trial_test_data2.csv')

In [6]:
train_df.head(3)

,Unnamed: 0,bResult,btPlayerRole,bjPlayerRole,bmPlayerRole,baPlayerRole,bsPlayerRole,rtPlayerRole,rjPlayerRole,rmPlayerRole,...,raPlayerChampion,rsPlayerChampion,bCoopPlayer,rCoopPlayer,bCoopChampion,rCoopChampion,vsPlayer,vsChampion,bTeamColor,rTeamColor
0,1,1.0,0.701884,-0.205048,0.975121,0.714486,0.685246,-0.001432,0.401764,-0.109146,...,0.935151,0.706502,0.591416,0.150967,-0.427561,1.361532,0.282831,0.413140,1.050299,0.494876
1,2,0.0,-0.233737,-0.483903,-0.007889,-0.728082,-0.667611,-0.052322,-1.655595,-0.996363,...,-0.138737,0.847053,-0.181804,-0.824227,-1.419230,0.368822,-1.585880,-0.253838,-0.687879,-0.464691
2,3,1.0,-0.795546,-0.009849,0.719975,-0.010759,-0.228221,0.993817,0.401764,0.023766,...,-0.138737,0.853285,-0.095593,-0.556974,0.226590,-0.737584,0.517496,1.724576,-0.863766,0.169911


In [8]:
# drop_cols = list(train_df.filter(regex='ChampionRole$').columns)
drop_cols = ['Unnamed: 0']

In [10]:
train_df = train_df.drop(drop_cols, axis=1)
train_df.columns

Index(['bResult', 'btPlayerRole', 'bjPlayerRole', 'bmPlayerRole',
       'baPlayerRole', 'bsPlayerRole', 'rtPlayerRole', 'rjPlayerRole',
       'rmPlayerRole', 'raPlayerRole', 'rsPlayerRole', 'btPlayerChampion',
       'bjPlayerChampion', 'bmPlayerChampion', 'baPlayerChampion',
       'bsPlayerChampion', 'rtPlayerChampion', 'rjPlayerChampion',
       'rmPlayerChampion', 'raPlayerChampion', 'rsPlayerChampion',
       'bCoopPlayer', 'rCoopPlayer', 'bCoopChampion', 'rCoopChampion',
       'vsPlayer', 'vsChampion', 'bTeamColor', 'rTeamColor'],
      dtype='object')

In [12]:
len(list(train_df.columns))

29

In [14]:
test_df = test_df.drop(drop_cols, axis=1)
len(list(test_df.columns))

29

In [16]:
y_train = train_df.loc[:, 'bResult']
X_train = train_df.loc[:, train_df.columns != 'bResult']
y_test = test_df.loc[:, 'bResult']
X_test = test_df.loc[:, test_df.columns != 'bResult']

In [18]:
X_train.count

<bound method DataFrame.count of       btPlayerRole  bjPlayerRole  bmPlayerRole  baPlayerRole  bsPlayerRole  \
0         0.701884     -0.205048      0.975121      0.714486      0.685246   
1        -0.233737     -0.483903     -0.007889     -0.728082     -0.667611   
2        -0.795546     -0.009849      0.719975     -0.010759     -0.228221   
3        -0.016062      0.500671      0.573369     -0.430835      0.776097   
4        -0.016062      0.390423      0.125801      0.932329      0.377655   
...            ...           ...           ...           ...           ...   
6819      0.741770      0.426780     -1.679006     -0.455187     -1.107000   
6820     -1.234005     -0.673525     -0.676336     -1.714401     -0.374685   
6821     -1.301891     -0.063371     -3.350123     -3.418044     -2.480091   
6822      0.227527      0.280107      0.988918     -0.612045     -0.008527   
6823     -3.426304     -3.328228     -2.321743     -2.369648     -1.656237   

      rtPlayerRole  rjPlayerRo

In [20]:
gnb = GaussianNB()

In [22]:
y_train.dtypes

dtype('float64')

In [24]:
gnb.fit(X_train, y_train)

GaussianNB()

In [26]:
# apparently the model must be fit prior to calling this
priors = gnb.predict_proba(X_test)
priors

array([[1.09464730e-01, 8.90535270e-01],
       [8.18111257e-01, 1.81888743e-01],
       [4.15942302e-01, 5.84057698e-01],
       ...,
       [4.19597512e-04, 9.99580402e-01],
       [9.18705834e-03, 9.90812942e-01],
       [1.08917211e-04, 9.99891083e-01]])

In [28]:
y_predict = gnb.predict(X_test)
mean_accuracy = gnb.score(X_test, y_test)
mean_accuracy

0.6596306068601583

In [30]:
accuracy = accuracy_score(y_test, y_predict)
print(accuracy)

0.6596306068601583


In [32]:
len(priors)

758

In [34]:
# tenfold training
rkf = RepeatedKFold(n_repeats=5, n_splits=10, random_state=3)

In [36]:
import numpy as np

In [38]:
X_array = X_train.to_numpy()
y_array = y_train.to_numpy()

In [40]:
accuracies = []

In [42]:
best_probs = {'prob_blue': 0, 'prob_red': 0}

In [44]:
best_accuracy = 0

In [46]:
for train_index, test_index in rkf.split(X_train):
    # print(f'    Train: index={train_index}')
    # print(f'    Test: index={test_index}')
    X_fold_train, X_fold_test = X_train.iloc[train_index], X_train.iloc[test_index]
    y_fold_train, y_fold_test = y_train.iloc[train_index], y_train.iloc[test_index]
    prob_blue = len([x for x in list(y_fold_train) if x == 1]) / len(list(y_fold_train))
    prob_red = len([x for x in list(y_fold_train) if x == 0]) / len(list(y_fold_train))
    print(f'probability that blue wins: {prob_blue}')
    print(f'probability that red wins: {prob_red}')
    model = GaussianNB()
    model.fit(X_fold_train, y_fold_train)
    fold_prediction = model.predict(X_fold_test)
    accuracy = accuracy_score(y_fold_test, fold_prediction)
    accuracies.append(accuracy)
    print(f'accuracy: {accuracy}')
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_probs['prob_blue'] = prob_blue
        best_probs['prob_red'] = prob_red

probability that blue wins: 0.548282038755903
probability that red wins: 0.4517179612440971
accuracy: 0.8184480234260615
probability that blue wins: 0.5443738804754926
probability that red wins: 0.45562611952450743
accuracy: 0.7891654465592972
probability that blue wins: 0.5474678391141508
probability that red wins: 0.4525321608858492
accuracy: 0.7862371888726208
probability that blue wins: 0.5484448786842534
probability that red wins: 0.4515551213157466
accuracy: 0.8067349926793558
probability that blue wins: 0.5488440247476392
probability that red wins: 0.4511559752523608
accuracy: 0.7683284457478006
probability that blue wins: 0.5468902637577336
probability that red wins: 0.45310973624226636
accuracy: 0.7859237536656891
probability that blue wins: 0.5444480625203517
probability that red wins: 0.45555193747964834
accuracy: 0.8225806451612904
probability that blue wins: 0.549820905242592
probability that red wins: 0.450179094757408
accuracy: 0.7976539589442815
probability that blue wi

In [48]:
accuracies

[0.8184480234260615,
 0.7891654465592972,
 0.7862371888726208,
 0.8067349926793558,
 0.7683284457478006,
 0.7859237536656891,
 0.8225806451612904,
 0.7976539589442815,
 0.7903225806451613,
 0.781524926686217,
 0.8096632503660323,
 0.7979502196193266,
 0.7994143484626647,
 0.7730600292825769,
 0.7903225806451613,
 0.781524926686217,
 0.7932551319648093,
 0.7903225806451613,
 0.7741935483870968,
 0.8343108504398827,
 0.8155197657393851,
 0.7920937042459737,
 0.7920937042459737,
 0.7994143484626647,
 0.7536656891495601,
 0.8064516129032258,
 0.8093841642228738,
 0.7903225806451613,
 0.7859237536656891,
 0.7903225806451613,
 0.7833089311859444,
 0.8023426061493412,
 0.7745241581259151,
 0.8008784773060029,
 0.8269794721407625,
 0.7741935483870968,
 0.7947214076246334,
 0.7903225806451613,
 0.7785923753665689,
 0.8123167155425219,
 0.7950219619326501,
 0.7584187408491947,
 0.8038067349926794,
 0.7906295754026355,
 0.8035190615835777,
 0.7917888563049853,
 0.8167155425219942,
 0.793255131964

In [50]:
len(accuracies)

50

In [52]:
sum(accuracies) / 50

0.7940208584689763

In [54]:
best_accuracy

0.8343108504398827

In [56]:
best_probs

{'prob_blue': 0.5483555845001629, 'prob_red': 0.4516444154998372}

In [58]:
gnb = GaussianNB(priors=[best_probs['prob_red'], best_probs['prob_blue']])

In [60]:
gnb.fit(X_train, y_train)

GaussianNB(priors=[0.4516444154998372, 0.5483555845001629])

In [62]:
tuned_prediction = gnb.predict(X_test)

In [64]:
accuracy_w_priors = accuracy_score(y_test, tuned_prediction)
accuracy_w_priors

0.6596306068601583